Libraries

In [88]:
import asyncio
import json
import pandas as pd
from typing import List, Dict, Literal
from httpx import AsyncClient, Response
from parsel import Selector
import mysql.connector
import numpy as np

Define cities to scrape

In [89]:
cities_to_scrape = [
    "Geneva"
]

"""
cities_to_scrape = [
    "Zurich", "Geneva", "Basel", "Lausanne", "Bern", "Winterthur", "Lucerne", "St-Gallen", "Lugano", "Biel", "Interlaken",
    "Thun", "Koeniz", "La-Chaux-de-Fonds", "Schaffhausen", "Fribourg", "Vernier", "Chur", "Neuchatel", "Uster", "Sion",
    "Lancy", "Emmen", "Yverdon-les-Bains", "Zug", "Kriens", "Rapperswil-Jona", "Duebendorf", "Montreux", "Frauenfeld", "Carouge",
    "Wetzikon", "Wil", "Renens", "Herisau", "Bellinzona", "Kloten", "Pratteln", "Allschwil", "Vevey", "Meyrin", "Monthey",
    "Onex", "Zollikon", "Waedenswil", "Kuesnacht", "Baden", "Weinfelden", "Sierre", "Le Locle", "Littau", "Morges", "Val-de-Travers",
    "Pully", "Seebach", "Ecublens", "Wettingen", "Adliswil", "Oberwil", "Nyon", "Ittigen", "Burgdorf", "Le-Grand-Saconnex",
    "Baar", "Muttenz", "Martigny", "Grenchen", "Buelach", "Rueti", "Thalwil", "Regensdorf", "Sierre", "Gossau", "Ascona",
    "Olten", "Aarau", "Horgen", "Schlieren", "Opfikon", "Sitten", "Liestal", "Chene-Bougeries", "Plan-les-Ouates", "Mendrisio",
    "Renens", "Langenthal", "Kreuzlingen", "Wohlen", "Steffisburg", "Moehlin", "Binningen", "Volketswil", "Locarno"
]
"""

'\ncities_to_scrape = [\n    "Zurich", "Geneva", "Basel", "Lausanne", "Bern", "Winterthur", "Lucerne", "St-Gallen", "Lugano", "Biel", "Interlaken",\n    "Thun", "Koeniz", "La-Chaux-de-Fonds", "Schaffhausen", "Fribourg", "Vernier", "Chur", "Neuchatel", "Uster", "Sion",\n    "Lancy", "Emmen", "Yverdon-les-Bains", "Zug", "Kriens", "Rapperswil-Jona", "Duebendorf", "Montreux", "Frauenfeld", "Carouge",\n    "Wetzikon", "Wil", "Renens", "Herisau", "Bellinzona", "Kloten", "Pratteln", "Allschwil", "Vevey", "Meyrin", "Monthey",\n    "Onex", "Zollikon", "Waedenswil", "Kuesnacht", "Baden", "Weinfelden", "Sierre", "Le Locle", "Littau", "Morges", "Val-de-Travers",\n    "Pully", "Seebach", "Ecublens", "Wettingen", "Adliswil", "Oberwil", "Nyon", "Ittigen", "Burgdorf", "Le-Grand-Saconnex",\n    "Baar", "Muttenz", "Martigny", "Grenchen", "Buelach", "Rueti", "Thalwil", "Regensdorf", "Sierre", "Gossau", "Ascona",\n    "Olten", "Aarau", "Horgen", "Schlieren", "Opfikon", "Sitten", "Liestal", "Chene-Bougerie

Create async client instance and set headers

In [90]:
client = AsyncClient(
    headers={
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "Accept-Language": "en-US,en;q=0.9",
    }
)

Parse listing function

In [91]:
def parse_next_data(response: Response) -> Dict:
    """Parse listing data from homegate search"""
    selector = Selector(response.text)
    next_data = selector.xpath("//script[contains(text(),'window.__INITIAL_STATE__')]/text()").get()
    if not next_data:
        return {}
    next_data_json = json.loads(next_data.strip("window.__INITIAL_STATE__="))
    return next_data_json

Scrape search function

In [92]:
async def scrape_search(city: str, query_type: Literal["rent", "buy"] = "buy") -> List[Dict]:
    """Scrape listing data from homegate search pages for a specific city"""
    url = f"https://www.homegate.ch/{query_type}/real-estate/city-{city}/matching-list"
    first_page = await client.get(url)
    data = parse_next_data(first_page)["resultList"]["search"]["fullSearch"]["result"]
    if data is None:
        print(f"No data found for {city}")
        return []
    search_data = data["listings"]
    max_search_pages = data["pageCount"]
    print(f"Scraped first search page for {city}, remaining ({max_search_pages} search pages)")
    other_pages = [client.get(url=str(first_page.url) + f"?ep={page}") for page in range(2, max_search_pages + 1)]
    for response in asyncio.as_completed(other_pages):
        data = parse_next_data(await response)
        if data is not None:
            search_data.extend(data["resultList"]["search"]["fullSearch"]["result"]["listings"])
            print(f"Scraped {len(search_data)} property listings from {city} search")
        await asyncio.sleep(5)  # delay
    return search_data

Main function

In [93]:
async def main():
    all_search_data = []
    for city in cities_to_scrape:
        city_search_data = await scrape_search(city, query_type="buy")
        all_search_data.extend(city_search_data)
    
    # Close client
    await client.aclose()

    json_string = json.dumps(all_search_data, indent=2)
    json_data = json.loads(json_string)

    # Specify the columns to include in the DataFrame
    columns = ['id', 'listing']

    # Extract the 'listing' data
    listing_data = [item['listing'] for item in json_data]

    # Extract the 'prices' data
    prices_data = [{'id': item['id'], **item['listing']['prices']['buy'], **item['listing']['address'], **(item['listing']['characteristics'] if 'characteristics' in item['listing'] else {}), **item['listing']['address']['geoCoordinates']} for item in json_data if 'prices' in item['listing'] and 'buy' in item['listing']['prices']]

    # Create the DataFrame
    df = pd.DataFrame(listing_data)
    # Create the DataFrame
    df_prices = pd.DataFrame(prices_data)
    # Replace 'NaN' values with None
    df_prices = df_prices.replace({np.nan: None})

    # Set the index of the DataFrame to the 'id' column
    df.set_index('id', inplace=True)
    # print(df_prices)

    #Write data to database
    # Create a connection to the database
    db = mysql.connector.connect(
        host="localhost",
        user="user",
        password="password",
        port="3306",
        database="realestatepredictor"
    )

    # Create a cursor object
    cursor = db.cursor()

    # Convert the DataFrame to a list of dictionaries
    data_dict = df_prices.to_dict('records')

    # Clear the table before inserting the new data. Uncomment code below to clear the table before inserting new data
    # cursor.execute("DELETE FROM homegate")
    # db.commit()
    # print("All entries deleted from 'homegate'.")

    # Iterate over the list of dictionaries and insert each one into the MySQL table
    for data in data_dict:
        query = "INSERT INTO homegate (homegateid, price, rooms, floor, livingSpace, street, latitude, longitude, locality, postalcode) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        values = (data['id'], data['price'], data['numberOfRooms'], data['floor'], data['livingSpace'], data['street'], data['latitude'], data['longitude'], data['locality'], data['postalCode'])
        cursor.execute(query, values)

    # Commit the transaction
    db.commit()

    # Close the cursor and connection
    cursor.close()
    db.close()

Run main function

In [94]:
if __name__ == "__main__":
    # asyncio.run(main())
    await main()

Scraped first search page for Geneva, remaining (18 search pages)
Scraped 30 property listings from Geneva search
Scraped 50 property listings from Geneva search
Scraped 70 property listings from Geneva search
Scraped 90 property listings from Geneva search
Scraped 110 property listings from Geneva search
Scraped 130 property listings from Geneva search
Scraped 150 property listings from Geneva search
Scraped 170 property listings from Geneva search
Scraped 190 property listings from Geneva search
Scraped 210 property listings from Geneva search
Scraped 230 property listings from Geneva search
Scraped 250 property listings from Geneva search
Scraped 270 property listings from Geneva search
Scraped 290 property listings from Geneva search
Scraped 310 property listings from Geneva search
Scraped 330 property listings from Geneva search
Scraped 350 property listings from Geneva search


KeyError: 'prices'